In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2022년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,category,tokenize_title,tokenize_content
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,IT/과학,"['티', '타다', '연말', '공세', '찻잔', '속', '태풍', '카카오'...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오..."
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'..."
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['내일', '방역', '패스', '유효', '기간', '네이버', '카카오', '...","['내일', '코로나', '방역', '패스', '유효', '기간', '적용', '된..."
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,경제,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '투자', '기술', '협력'..."
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,IT/과학,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시..."
...,...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['팬데믹', '엔데믹', '카카오', '내비', '결산', '데이터', '공개']","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요..."
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지..."
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ..."
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['외양간', '고치는', '카카오', '사과', '보상', '모범', '답안', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '..."


In [4]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [5]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['티', '타다', '연말', '공세', '찻잔', '속', '태풍', '카카오', '모방', '한계'],
 ['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '재도전'],
 ['내일', '방역', '패스', '유효', '기간', '네이버', '카카오', '업데이트'],
 ['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래', '찾는', '빅', '테크'],
 ['카카오', '이제', '팬', '시대', '한국', '주도할'],
 ['네이버', '카카오', '새로운', '격전', '지는', '디지털', '헬'],
 ['개장', '새해', '코스피', '코스닥', '상승', '출발', '카겜', '위메이드'],
 ['코스피', '효과', '대감', '솔솔', '삼성', '전자', '카카오', '오름세'],
 ['신년사', '카카오', '팬', '시대', '왔다', '한국', '우위', '가질'],
 ['삼성', '전자', '이재용', '국내', '주식', '부호', '카카오', '김범수'],
 ['신년사', '카카오', '게임', '대표', '비욘드', '게임', '보여줄', '해'],
 ['신년사', '카카오', '팬', '시대', '도래', '산업', '도약', '기회'],
 ['신년사', '카카오', '게임', '대표', '올해', '비욘드', '게임', '보여주는', '해'],
 ['신년사', '카카오', '게임', '대표', '비욘드', '게임', '본격', '적', '보여줄'],
 ['신년사', '카카오', '게임', '대표', '비욘드', '게임', '진가', '보여줄'],
 ['카카오', '게임', '대표', '메타', '버스', '스포츠', '적극', '확장'],
 ['카카오', '게임', '비욘드', '게임', '확장', '박차'],
 ['신년사', '카카오', '게임', '대표', '비욘드', '게임', '확장', '박차'],
 ['신년사', '카카오', '게임', '대표', '

In [6]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<8248 unique tokens: ['공세', '모방', '속', '연말', '찻잔']...>


In [7]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3248 꿈틀
3249 자신있다
3250 전원
3251 고위
3252 내놔
3253 전직
3254 읽는
3255 효은
3256 긍정
3257 신호
3258 착오
3259 단어
3260 번역기
3261 오염시킨
3262 페미니즘
3263 자신있어
3264 바꾸는
3265 였다
3266 기아
3267 몰려
3268 슈퍼
3269 돌아가겠다
3270 챙긴
3271 첫날
3272 보여준다
3273 박정호
3274 손잡다
3275 열었다
3276 젊어지는
3277 이끄는
3278 주주총회
3279 통과
3280 넘어선다
3281 조화
3282 갖춘
3283 보람
3284 함께한
3285 앱스토어
3286 청
3287 넘긴
3288 바톤
3289 갖게
3290 물러나는
3291 안정감
3292 르포
3293 아시나요
3294 오름
3295 제주도
3296 출항
3297 잇는다
3298 위원회
3299 미디어
3300 개막
3301 뭉쳤다
3302 용사
3303 열렸다
3304 올랐다
3305 바꿨다
3306 빼고
3307 달라질까
3308 뚜비
3309 망월
3310 수성구
3311 쏟아낸다
3312 어린이
3313 담화
3314 전통주
3315 대동
3316 이륜차
3317 전기
3318 총서
3319 부여
3320 드는
3321 얻을까
3322 결의
3323 섭
3324 진웅
3325 주류
3326 인방
3327 지배
3328 키웠다
3329 의결
3330 기업가
3331 된다며
3332 미니
3333 손댄다
3334 체질개선
3335 소수점
3336 담보
3337 국토부
3338 권한
3339 방통위
3340 비율
3341 네트워크
3342 샌드박스
3343 협업
3344 규합
3345 거버넌스
3346 리즘
3347 하이퍼
3348 왕좌
3349 새로
3350 포함
3351 경기도
3352 놀자
3353 야
3354 두기
3355 랩스
3356 케어
3357 그대로
3358 번호
3359 아이폰
33

In [8]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1)],
 [(5, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)],
 [(5, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)],
 [(5, 1),
  (19, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1)],
 [(5, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)],
 [(5, 1), (19, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)],
 [(42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(5, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1)],
 [(5, 1), (32, 1), (35, 1), (36, 1), (56, 1), (57, 1), (58, 1), (59, 1)],
 [(5, 1), (51, 1), (54, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1)],
 [(5, 1), (16, 1), (57, 1), (65, 2), (66, 1), (67, 1), (68, 1)],
 [(5, 1), (32, 1), (35, 1), (57, 1), (69, 1), (70, 1), (71, 1), (72, 1)],
 [(5, 1), (16, 1), (57, 1), (65, 2), (66, 1), (68, 1), (73, 1), (74, 1)],
 [(5, 1), 

In [9]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.023*"사태" + 0.018*"관련" + 0.016*"대표" + 0.014*"먹통" + 0.013*"대리" + 0.012*"사퇴" + 0.012*"현장" + 0.009*"급등" + 0.008*"포토" + 0.008*"운전"
1 : 0.047*"장애" + 0.019*"공정위" + 0.017*"서비스" + 0.016*"접수" + 0.012*"당" + 0.011*"착수" + 0.011*"피해" + 0.010*"제재" + 0.008*"제한" + 0.008*"화재"
2 : 0.025*"센터" + 0.025*"데이터" + 0.023*"방지법" + 0.017*"대책" + 0.015*"먹통" + 0.009*"그룹" + 0.009*"이니셔티브" + 0.009*"기자회견" + 0.009*"미래" + 0.008*"이중화"
3 : 0.021*"이용자" + 0.019*"최태원" + 0.013*"회사" + 0.012*"사례" + 0.009*"불편" + 0.008*"컨콜" + 0.007*"카톡" + 0.007*"수습" + 0.006*"결론" + 0.006*"먹통"
4 : 0.046*"보상" + 0.036*"피해" + 0.017*"사과" + 0.015*"사태" + 0.014*"무료" + 0.014*"점검" + 0.014*"서비스" + 0.013*"정상화" + 0.011*"대표" + 0.011*"판교"
5 : 0.025*"영업" + 0.025*"공인" + 0.023*"소상" + 0.013*"게임" + 0.012*"먹통" + 0.011*"안보" + 0.008*"코스피" + 0.008*"뱅크" + 0.008*"이익" + 0.007*"상승"
6 : 0.044*"사태" + 0.018*"독과점" + 0.017*"먹통" + 0.013*"재발" + 0.011*"사고" + 0.011*"방지" + 0.011*"사과" + 0.010*"사진" + 0.009*"마련" + 0.008*"국감"
7 : 0.031*"숙인" + 0.013*"먹통" + 0.012*"대표" + 0.011*"호출" + 0.010

In [10]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.026665894),
 (1, 0.026663207),
 (2, 0.67600876),
 (3, 0.026661433),
 (4, 0.026661491),
 (5, 0.026661394),
 (6, 0.026661394),
 (7, 0.11069302),
 (8, 0.026661564),
 (9, 0.026661858)]

In [11]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.032477867), (1, 0.03247744), (2, 0.032480426), (3, 0.16989626), (4, 0.032481145), (5, 0.5702675), (6, 0.0324774), (7, 0.032482952), (8, 0.03248097), (9, 0.03247805)]
[(0, 0.030466318), (1, 0.03046236), (2, 0.030472344), (3, 0.030465353), (4, 0.14923643), (5, 0.6070428), (6, 0.03046174), (7, 0.030465912), (8, 0.030464457), (9, 0.030462278)]
[(0, 0.032501016), (1, 0.032500643), (2, 0.032501806), (3, 0.032500777), (4, 0.17829165), (5, 0.56169873), (6, 0.032500595), (7, 0.032502085), (8, 0.032501515), (9, 0.032501195)]
[(0, 0.03046639), (1, 0.030462392), (2, 0.030472461), (3, 0.030465417), (4, 0.14906566), (5, 0.60721326), (6, 0.030461764), (7, 0.03046593), (8, 0.03046448), (9, 0.030462302)]
[(0, 0.027852144), (1, 0.027851086), (2, 0.20052516), (3, 0.027851734), (4, 0.18746378), (5, 0.21307018), (6, 0.027851967), (7, 0.02785696), (8, 0.23182571), (9, 0.027851278)]
[(0, 0.029425282), (1, 0.029423555), (2, 0.029428067), (3, 0.029424628), (4, 0.18325

In [12]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,IT/과학,"['티', '타다', '연말', '공세', '찻잔', '속', '태풍', '카카오'...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오...",2.0,0.676006
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'...",8.0,0.732526
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['내일', '방역', '패스', '유효', '기간', '네이버', '카카오', '...","['내일', '코로나', '방역', '패스', '유효', '기간', '적용', '된...",4.0,0.738560
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,경제,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '투자', '기술', '협력'...",6.0,0.357720
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,IT/과학,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시...",3.0,0.713914
...,...,...,...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['팬데믹', '엔데믹', '카카오', '내비', '결산', '데이터', '공개']","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요...",6.0,0.557479
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",5.0,0.608870
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",4.0,0.266831
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['외양간', '고치는', '카카오', '사과', '보상', '모범', '답안', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '...",6.0,0.347943


In [13]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([2., 8., 4., 6., 3., 7., 5., 1., 0., 9.])

In [14]:
df['topic(title)'].value_counts()

4.0    1411
2.0    1224
7.0    1021
0.0    1013
9.0     955
8.0     946
5.0     903
6.0     806
3.0     722
1.0     583
Name: topic(title), dtype: int64

In [15]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [16]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<43345 unique tokens: ['가로막혀', '가맹', '가운데', '가장', '가파르']...>


In [17]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
38345 선로
38346 세세히
38347 쓰여졌다라는
38348 쓰였다
38349 아니다라는
38350 없어졌을
38351 올라가기는
38352 옮긴다고
38353 왔었고
38354 유용했냐는
38355 줄었는지
38356 토장
38357 파다
38358 회복하도록
38359 깨졌을까
38360 데을
38361 두되
38362 두었을
38363 밀접히
38364 안전하지만
38365 않았는가
38366 애매하게
38367 엄청나다
38368 여긴다
38369 연소
38370 왠만한
38371 이어가도록
38372 지었고
38373 톡임
38374 해하
38375 거둬들였다
38376 돌아서서
38377 빈번히
38378 추세라면
38379 적막
38380 내놓는가
38381 돌아왔을
38382 마시기
38383 만났고
38384 맞던
38385 여겨지
38386 풍랑
38387 휘하며
38388 가졌다고
38389 재야
38390 축전지
38391 일어서고
38392 파헤치겠다
38393 남았나요
38394 들려
38395 매만지고
38396 이뤄져야겠다는
38397 어떨지
38398 꽂아
38399 외산
38400 해보시죠
38401 가능한가요
38402 가연
38403 가우
38404 가우스
38405 가지고서
38406 가트너
38407 같기는
38408 거여
38409 건지는
38410 괜찮은지
38411 구어
38412 금품
38413 나가더라도
38414 나네요
38415 나누는데
38416 나오고는
38417 내연기관차
38418 넓지
38419 넘는다는
38420 놓죠
38421 닛산
38422 대단하네요
38423 대부터는
38424 되겠는데요
38425 되기로
38426 뒀고요
38427 드는군요
38428 드렸는데
38429 들으니까
38430 리즌
38431 마는
38432 만들어지다
38433 만들었고요
38434 많더라고요
38435 말씀드렸는데
38436 모라
384

In [18]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.002*"사무실" + 0.002*"클레이튼" + 0.001*"무제" + 0.001*"회의실" + 0.001*"원격" + 0.001*"매출" + 0.001*"게임" + 0.001*"재택" + 0.001*"근무" + 0.001*"브레인"
1 : 0.004*"최태원" + 0.004*"재난" + 0.004*"비상" + 0.004*"사이버" + 0.003*"방통위" + 0.003*"금감원" + 0.003*"간사" + 0.003*"신용" + 0.003*"원장" + 0.003*"먹통"
2 : 0.002*"병원" + 0.001*"증상" + 0.001*"물류" + 0.001*"의료" + 0.001*"별세" + 0.001*"장례식" + 0.001*"환자" + 0.001*"배송" + 0.001*"모" + 0.001*"진료"
3 : 0.007*"매출" + 0.006*"게임" + 0.005*"주가" + 0.005*"뱅크" + 0.004*"증권" + 0.004*"하락" + 0.004*"거래" + 0.004*"영업" + 0.004*"감소" + 0.004*"동기"
4 : 0.006*"피해" + 0.005*"데이터" + 0.005*"센터" + 0.004*"보상" + 0.004*"장애" + 0.004*"서비스" + 0.004*"화재" + 0.003*"사태" + 0.003*"판교" + 0.003*"복구"
5 : 0.006*"택시" + 0.005*"의결권" + 0.005*"공정위" + 0.005*"먹통" + 0.004*"큐브" + 0.004*"대리" + 0.003*"모빌리티" + 0.003*"호출" + 0.003*"독과점" + 0.003*"심야"
6 : 0.001*"구글" + 0.001*"아웃" + 0.001*"방통위" + 0.001*"라씨" + 0.001*"주무" + 0.001*"화하면서" + 0.001*"링크" + 0.001*"결제" + 0.001*"타이밍" + 0.001*"인정"
7 : 0.007*"기사" + 0.005*"섹션" + 0.005*"분류" + 0.005*"안내" + 

In [20]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(3, 0.43275234), (4, 0.50444067)]

In [21]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.016844578), (1, 0.60154516), (2, 0.016844256), (3, 0.26368833), (4, 0.01685313), (5, 0.016846994), (6, 0.016844211), (7, 0.016845327), (8, 0.016844086), (9, 0.016843932)]
[(0, 0.016045654), (1, 0.56707734), (2, 0.016045615), (3, 0.3045262), (4, 0.016066486), (5, 0.016048258), (6, 0.016053451), (7, 0.016046416), (8, 0.016045373), (9, 0.016045228)]
[(0, 0.016042244), (1, 0.52236104), (2, 0.016042193), (3, 0.3492599), (4, 0.016075274), (5, 0.016043788), (6, 0.01604899), (7, 0.016042868), (8, 0.016041936), (9, 0.016041813)]
[(0, 0.016139079), (1, 0.47769296), (2, 0.016138991), (3, 0.39313993), (4, 0.016185407), (5, 0.016140686), (6, 0.01614597), (7, 0.016139658), (8, 0.016138718), (9, 0.016138598)]
[(0, 0.0161574), (1, 0.44308788), (2, 0.0161572), (3, 0.42761362), (4, 0.016184174), (5, 0.016163873), (6, 0.01615719), (7, 0.01616471), (8, 0.016157033), (9, 0.0161569)]
[(0, 0.01540725), (1, 0.27950624), (2, 0.015407088), (3, 0.59718806), (4, 0.015432

In [22]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,IT/과학,"['티', '타다', '연말', '공세', '찻잔', '속', '태풍', '카카오'...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오...",2.0,0.676006,4.0,0.504283
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'...",8.0,0.732526,4.0,0.637316
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['내일', '방역', '패스', '유효', '기간', '네이버', '카카오', '...","['내일', '코로나', '방역', '패스', '유효', '기간', '적용', '된...",4.0,0.738560,4.0,0.860919
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,경제,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '투자', '기술', '협력'...",6.0,0.357720,4.0,0.651520
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,IT/과학,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시...",3.0,0.713914,4.0,0.899502
...,...,...,...,...,...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,IT/과학,"['팬데믹', '엔데믹', '카카오', '내비', '결산', '데이터', '공개']","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요...",6.0,0.557479,4.0,0.619079
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,IT/과학,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",5.0,0.608870,4.0,0.860263
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,사회,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",4.0,0.266831,4.0,0.867379
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,IT/과학,"['외양간', '고치는', '카카오', '사과', '보상', '모범', '답안', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '...",6.0,0.347943,4.0,0.855549


In [23]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([4., 3., 5., 7., 1., 6., 8., 2.])

In [24]:
df['topic(content)'].value_counts()

4.0    7311
3.0    1774
5.0     354
7.0      75
1.0      32
2.0      21
8.0      16
6.0       1
Name: topic(content), dtype: int64

In [25]:
# CSV 파일로 저장
csv_filename = "/content/drive/MyDrive/2023년 10월-11월 프로젝트/LDA모델 적용/news_2022.csv"
df.to_csv(csv_filename, index=False)